In [1]:
import os

os.environ["SEED"] = "0"
os.environ["TRAIN_DIR"] = "TRAIN"

import numpy as np

from src.aac.AAC import AAC
from src.constant import DATA_DIR, DATABASE_DIR
from src.database import DB
from src.database.queries import get_model_training_data, get_solvers_count
from src.instance.TSP_Instance import TSP_from_index_file
from src.solver.Portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.log import logger
from src.surrogate.SurrogatePolicy import SurrogatePolicy, TestSurrogatePolicy
from src.surrogate.wrapper import EmptyWrapper

In [2]:
train_instances = TSP_from_index_file(
    filepath=DATA_DIR / "TSP" / "TRAIN" / "index.json",
    cut_off_cost=100,
    cut_off_time=10,
    n=5,
)
test_instances = TSP_from_index_file(
    filepath=DATA_DIR / "TSP" / "TEST" / "index.json",
    cut_off_cost=1000,
    cut_off_time=100,
    n=250,
)

In [3]:
for instance in train_instances:
    instance.cut_off_time = round(10 * ((instance.n_cities / 600) ** 2.2), 2)
    instance.cut_off_cost = 10 * instance.cut_off_time

In [4]:
# surrogate_policy = SurrogatePolicy(
#     estimator_wrapper=EmptyWrapper(),
#     first_fit_solver_count=1,
#     refit_solver_count=2,
# )

surrogate_policy = TestSurrogatePolicy(
    estimator_wrapper=EmptyWrapper(),
    first_fit_solver_count=1,
    refit_solver_count=2,
)

In [5]:
SOLVERS_N = 2
ATTEMPTS_N = 3
MAX_ITER = 2

solvers = []
largest_marginal_contribution_solver = None

for solver_i in range(SOLVERS_N):
    logger.info(f"Solver {solver_i + 1}/{SOLVERS_N}")

    best_cost = np.inf
    best_solver = None
    attempt_solvers = []

    for attempt_i in range(ATTEMPTS_N):
        logger.info(f"Attempt {attempt_i + 1}/{ATTEMPTS_N}")

        if largest_marginal_contribution_solver is not None:
            new_solver = largest_marginal_contribution_solver.copy()
        else:
            new_solver = TSP_LKH_Solver()

        iteration_solvers = solvers + [new_solver]

        portfolio = Portfolio.from_iterable(iteration_solvers)
        aac = AAC(
            portfolio=portfolio,
            instance_list=train_instances,
            prefix=f"config;solver={solver_i+1};attempt={attempt_i+1}",
            max_iter=MAX_ITER,
            i=solver_i,
            calculate_features=False,
            surrogate_policy=surrogate_policy,
        )
        portfolio = aac.configure()
        result = portfolio.evaluate(  # fix cut-off times before validation
            instance_list=train_instances,
            prefix=f"validate;solver={solver_i+1};attempt={attempt_i+1}",
            cache=True,
        )
        attempt_solvers.append(portfolio[solver_i])
        logger.info(
            f"Attempt {attempt_i + 1}/{ATTEMPTS_N}: cost = {result.cost:.2f}"
        )
        if result.cost < best_cost:
            best_cost = result.cost
            best_solver = portfolio[solver_i]

    solvers.append(best_solver)
    logger.info(f"Solver {solver_i + 1}/{SOLVERS_N}: best cost = {best_cost:.2f}")

    if solver_i < SOLVERS_N - 1:
        largest_marginal_contribution_solver = None
        best_cost = np.inf
        for attempt_i, solver in enumerate(attempt_solvers):
            if solver != best_solver:
                portfolio = Portfolio.from_iterable(solvers + [solver])
                result = portfolio.evaluate(
                    instance_list=train_instances,
                    prefix=f"largest_marginal_contribution;attempt={attempt_i+1}",
                    cache=True,
                )
                if result.cost < best_cost:
                    best_cost = result.cost
                    largest_marginal_contribution_solver = solver

[2025-04-29 13:00:29] INFO      Solver 1/2
[2025-04-29 13:00:29] INFO      Attempt 1/3
[2025-04-29 13:00:29] DEBUG     AAC(prefix=config;solver=1;attempt=1, iter=1/2)
[2025-04-29 13:00:29] DEBUG     SurrogatePolicy(estimator_wrapper=<src.surrogate.wrapper.wrapper.EmptyWrapper object at 0x0000019EFFD44700>, first_fit_solver_count=1, refit_solver_count=2, last_fit_solver_count=0, is_fitted=False)
[2025-04-29 13:00:29] DEBUG     SurrogatePolicy.notify_iter(iter=1, solver_count=0)
[2025-04-29 13:00:29] DEBUG     Portfolio.evaluate(config;solver=1;attempt=1;aac_iter=1)
[2025-04-29 13:00:29] DEBUG     Portfolio(size=1)[Solver(id=52980316141715150)]
[2025-04-29 13:00:29] DEBUG     solve(prefix=config;solver=1;attempt=1;aac_iter=1, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/TRAIN/cluster_netgen/000.tsp))
[2025-04-29 13:00:29] DEBUG     solve(prefix=config;solver=1;attempt=1;aac_iter=1, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/TRAIN/co

In [8]:
solver = portfolio[0]
instance = train_instances[0]
X = np.concatenate([solver.get_array(), instance.get_array()])
X = X.reshape(1, -1)
X.shape

(1, 23)

In [19]:
surrogate_policy.estimator_wrapper.predict(X, np.array([instance.cut_off_time]))

[2025-04-29 12:22:35] DEBUG     Predicting with empty model wrapper with 1 samples
[2025-04-29 12:22:35] DEBUG     Predicting with empty model with 1 samples


array([0.])

In [ ]:

cost = estimator_wrapper.predict(X, instance.cut_off_time)[0]

In [7]:
surrogate_policy.estimator_wrapper

In [20]:
DB().get_solvers()

,id,ASCENT_CANDIDATES,BACKBONE_TRIALS,BACKTRACKING,CANDIDATE_SET_TYPE,EXTRA_CANDIDATES,EXTRA_CANDIDATE_SET_TYPE,GAIN23,GAIN_CRITERION,INITIAL_STEP_SIZE,...,MAX_BREADTH,MAX_CANDIDATES,MOVE_TYPE,PATCHING_A,PATCHING_C,POPULATION_SIZE,RESTRICTED_SEARCH,SUBGRADIENT,SUBSEQUENT_MOVE_TYPE,SUBSEQUENT_PATCHING
0,52980316141715150,0.60,1.0,0.0,0.0,0.7,0.0,1.0,0.0,0.0,...,0.444643,1.000000,0.25,0.6,0.2,0.061224,0.0,0.0,5.0,1.0
1,287011037061967493,0.75,0.0,0.0,0.0,0.3,0.0,1.0,0.0,0.0,...,0.592453,0.777778,0.75,0.8,0.2,0.836735,1.0,1.0,3.0,1.0
2,473831520656465062,0.25,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.5,...,0.470388,0.888889,1.00,0.6,0.2,0.244898,0.0,1.0,3.0,1.0
3,2058572598820127428,0.90,1.0,0.0,3.0,0.6,0.0,1.0,0.0,1.0,...,0.343259,0.555556,0.00,0.6,0.8,0.357143,1.0,1.0,5.0,0.0
4,176228367859327934,0.85,1.0,0.0,0.0,0.4,0.0,0.0,1.0,0.5,...,0.354664,0.333333,0.25,0.8,0.4,0.408163,1.0,1.0,2.0,0.0


In [4]:
db = DB(db_path=DATABASE_DIR / "test-2025_04_29_06_36_27.db")

In [5]:
X, y, cut_off = get_model_training_data(db)

In [6]:
cut_off.shape

(25,)

In [10]:
get_solvers_count(db)

5

In [ ]:
db.query2df(f"select count(*) from {DB.SCHEMA.SOLVERS}").iloc[0, 0]

,count(*)
0,5


: 

In [4]:
query = f"""
select 
    {db.SCHEMA.RESULTS}.cost,
    {db.SCHEMA.RESULTS}.cut_off_time,
    {db.SCHEMA.SOLVERS}.*,
    {db.SCHEMA.INSTANCES}.*
from {db.SCHEMA.RESULTS}
join {db.SCHEMA.INSTANCES} on {db.SCHEMA.RESULTS}.instance_id = {db.SCHEMA.INSTANCES}.id
join {db.SCHEMA.SOLVERS} on {db.SCHEMA.RESULTS}.solver_id = {db.SCHEMA.SOLVERS}.id
where {db.SCHEMA.RESULTS}.cached = 0 and {db.SCHEMA.RESULTS}.surrogate = 0
"""
df = db.query2df(query)

In [6]:
df = df.drop(columns=["id", "filepath", "optimum"])
df = df.dropna()


In [7]:
df

,cost,cut_off_time,ASCENT_CANDIDATES,BACKBONE_TRIALS,BACKTRACKING,CANDIDATE_SET_TYPE,EXTRA_CANDIDATES,EXTRA_CANDIDATE_SET_TYPE,GAIN23,GAIN_CRITERION,...,MAX_BREADTH,MAX_CANDIDATES,MOVE_TYPE,PATCHING_A,PATCHING_C,POPULATION_SIZE,RESTRICTED_SEARCH,SUBGRADIENT,SUBSEQUENT_MOVE_TYPE,SUBSEQUENT_PATCHING
0,6.90,0.69,0.60,1.0,0.0,0.0,0.7,0.0,1.0,0.0,...,0.444643,1.000000,0.25,0.6,0.2,0.061224,0.0,0.0,5.0,1.0
1,4.40,0.44,0.60,1.0,0.0,0.0,0.7,0.0,1.0,0.0,...,0.444643,1.000000,0.25,0.6,0.2,0.061224,0.0,0.0,5.0,1.0
2,52.30,5.23,0.60,1.0,0.0,0.0,0.7,0.0,1.0,0.0,...,0.444643,1.000000,0.25,0.6,0.2,0.061224,0.0,0.0,5.0,1.0
3,2.68,3.04,0.60,1.0,0.0,0.0,0.7,0.0,1.0,0.0,...,0.444643,1.000000,0.25,0.6,0.2,0.061224,0.0,0.0,5.0,1.0
4,59.30,5.93,0.60,1.0,0.0,0.0,0.7,0.0,1.0,0.0,...,0.444643,1.000000,0.25,0.6,0.2,0.061224,0.0,0.0,5.0,1.0
5,4.40,0.44,0.75,0.0,0.0,0.0,0.3,0.0,1.0,0.0,...,0.592453,0.777778,0.75,0.8,0.2,0.836735,1.0,1.0,3.0,1.0
6,6.90,0.69,0.75,0.0,0.0,0.0,0.3,0.0,1.0,0.0,...,0.592453,0.777778,0.75,0.8,0.2,0.836735,1.0,1.0,3.0,1.0
7,1.65,3.04,0.75,0.0,0.0,0.0,0.3,0.0,1.0,0.0,...,0.592453,0.777778,0.75,0.8,0.2,0.836735,1.0,1.0,3.0,1.0
8,4.06,5.93,0.75,0.0,0.0,0.0,0.3,0.0,1.0,0.0,...,0.592453,0.777778,0.75,0.8,0.2,0.836735,1.0,1.0,3.0,1.0
9,52.30,5.23,0.75,0.0,0.0,0.0,0.3,0.0,1.0,0.0,...,0.592453,0.777778,0.75,0.8,0.2,0.836735,1.0,1.0,3.0,1.0


In [8]:
y = df["cost"].to_numpy()
cut_off = df["cut_off_time"].to_numpy()
y = np.where(y >= cut_off, cut_off, y)
X = df.drop(columns="cost").to_numpy()

In [9]:
y

array([0.69, 0.44, 5.23, 2.68, 5.93, 0.44, 0.69, 1.65, 4.06, 5.23, 0.44,
       0.69, 3.04, 5.23, 5.93, 0.15, 0.44, 0.69, 3.3 , 3.9 , 0.43, 0.69,
       3.04, 5.23, 5.93])

In [10]:
cut_off

array([0.69, 0.44, 5.23, 3.04, 5.93, 0.44, 0.69, 3.04, 5.93, 5.23, 0.44,
       0.69, 3.04, 5.23, 5.93, 3.04, 0.44, 0.69, 5.23, 5.93, 0.44, 0.69,
       3.04, 5.23, 5.93])

In [11]:
X.shape

(25, 24)